In [1]:
# import pandas as pd
from pyspark import SparkContext, SparkConf, HiveContext
from pyspark.sql import SparkSession, Row, functions, Column
from pyspark.sql.types import StringType 
import pyspark.sql.functions as F

# pd.set_option('display.max_columns',  30)

In [2]:
def create_config(app_name,
                  memory=1,
                  instances=1,
                  cores=2,
                  max_result_size=50,
                  driver_memory=1):
    """
    simple wrapper for SparkConfig().set()
    :param memory: int, max provided memory, in gigabytes
    :param cores: int, max cores
    :return: pyspark.conf.SparkConf object
    """
    spark_conf = SparkConf().setAppName(app_name)
#     spark_conf.set('spark.executor.memory', f'{memory}g')
#     spark_conf.set('spark.executor.instances', f'{instances}')
#     spark_conf.set('spark.executor.cores', f'{cores}')
#     spark_conf.set('spark.driver.memory', f'{driver_memory}g')
#     spark_conf.set("spark.executor.allowSparkContext", 'true')
#     spark_conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
#     spark_conf.set('hive.exec.dynamic.partition.mode','nonstrict')
    return spark_conf

def create_session(conf,):
    return SparkSession.Builder().config(conf=conf).getOrCreate()

ses = create_session(create_config('tmp_writer'))

In [3]:
ses.sparkContext

<SparkContext master=local[*] appName=tmp_writer>

In [4]:
file = ses.sparkContext.textFile('admin_lev6.txt')

In [5]:
file = file.map(lambda x: x.split(';'))

In [6]:
cols = file.take(1)[0]

In [7]:
file = file.filter(lambda x: x != cols)

In [8]:
df = ses.createDataFrame(file, schema=cols)

In [9]:
df.show(2)

+-------------------+-------------+-------------------+---------+--------+------+---------+--------+---------+--------+---------+--------------------+---------+-----------------+---------+--------+---------+--------+---------+--------+---------+--------+---------+--------+----------+---------+
|               NAME|      NAME_EN|            NAME_RU|ADMIN_LVL|OSM_TYPE|OSM_ID|ADMIN_L1D|ADMIN_L1|ADMIN_L2D|ADMIN_L2|ADMIN_L3D|            ADMIN_L3|ADMIN_L4D|         ADMIN_L4|ADMIN_L5D|ADMIN_L5|ADMIN_L6D|ADMIN_L6|ADMIN_L7D|ADMIN_L7|ADMIN_L8D|ADMIN_L8|ADMIN_L9D|ADMIN_L9|ADMIN_L10D|ADMIN_L10|
+-------------------+-------------+-------------------+---------+--------+------+---------+--------+---------+--------+---------+--------------------+---------+-----------------+---------+--------+---------+--------+---------+--------+---------+--------+---------+--------+----------+---------+
|      Чунский район|Chunsky Rayon|      Чунский район|        6|relation|190097|         |        |    60189|  Рос

In [6]:
import pandas as pd
d = pd.read_csv('admin_lev8.txt', sep=';')

In [7]:
# df = ses.read.options(delimiter=';', header=True).format('csv').load('admin_lev8.txt')
df = ses.createDataFrame(d.astype(str))
# # df_p = df.toPandas()

def last_nonnull_func(osm_ids):
    return '2'

# udf_cols = [F.col(f'ADMIN_L{i}D') for i in range(1, 11)]

last_nonnull_udf = F.udf(lambda osm_ids: last_nonnull_udf(osm_ids), StringType())

In [8]:
df.withColumn('par_osm_id', last_nonnull_udf(F.col('NAME'))).show()

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/home/ripper/spark-3.1.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/home/ripper/spark-3.1.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/ripper/spark-3.1.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 211, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/home/ripper/spark-3.1.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 132, in dump_stream
    for obj in iterator:
  File "/home/ripper/spark-3.1.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 200, in _batched
    for item in iterator:
  File "/home/ripper/spark-3.1.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 450, in mapper
 